In [1]:
import keras

In [2]:
from keras import layers

In [3]:
import numpy as np

In [4]:
import matplotlib.pyplot as plt

In [5]:
latent_dim = 28
height = 28
width = 28
channels = 3

In [6]:
generator_input = keras.Input(shape=(latent_dim,))

In [7]:
x = layers.Dense(128 * 14 * 14)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((14, 14, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)

2022-02-08 15:13:29.029322: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 15:13:29.062114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 15:13:29.062303: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 15:13:29.062837: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the app

In [8]:
generator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28)]              0         
                                                                 
 dense (Dense)               (None, 25088)             727552    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 25088)             0         
                                                                 
 reshape (Reshape)           (None, 14, 14, 128)       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 256)       819456    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 14, 14, 256)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 28, 28, 256)      104883

In [9]:
import tensorflow

In [10]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)
discriminator = tensorflow.keras.models.Model(discriminator_input, x)
discriminator_optimizer = tensorflow.keras.optimizers.RMSprop(learning_rate=0.0008,clipvalue=1.0,decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,loss='binary_crossentropy')


In [11]:
discriminator.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 3)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 26, 26, 128)       3584      
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 26, 26, 128)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 12, 12, 128)       262272    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 12, 12, 128)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 5, 5, 128)         262272    
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 5, 5, 128)         0   

In [12]:
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)
gan_optimizer = tensorflow.keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

/usr/lib/python3.10/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [13]:
gan.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28)]              0         
                                                                 
 model (Functional)          (None, 28, 28, 3)         5910787   
                                                                 
 model_1 (Functional)        (None, 1)                 790529    
                                                                 
Total params: 6,701,316
Trainable params: 5,910,787
Non-trainable params: 790,529
_________________________________________________________________


In [14]:
import os

In [15]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing import image
from tensorflow.keras.datasets import mnist

In [16]:
(train_x , train_y) , (test_x , test_y) = mnist.load_data()

In [17]:
train_x.shape

(60000, 28, 28)

In [18]:
train_x = train_x.reshape((-1,height,width,channels)).astype('float32') / 255

In [19]:
iterations = 10000

In [20]:
batch_size=20

In [21]:
save_dir = 'mnist'

In [ ]:
start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    generated_images = generator.predict(random_latent_vectors)
    stop = start + batch_size
    real_images = train_x[start:stop]
    
    combined_images = np.concatenate([generated_images,real_images])
    labels = np.concatenate([np.ones((batch_size,1)),
                            np.zeros((batch_size,1))])
    labels += 0.05 * discriminator.train_on_batch(combined_images , labels)
    d_loss = discriminator.train_on_batch(combined_images , labels)
    
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    misleading_targets = np.zeros((batch_size,1))
    a_loss = gan.train_on_batch(random_latent_vectors,misleading_targets)
    
    start += batch_size
    if start > len(train_x) - batch_size:
        start = 0
    if step % 100 == 0:
        gan.save_weights('gan-mnist.h5')
        
        print('discriminator loss', d_loss)
        print('adverserial loss' , a_loss)
        
        img = image.array_to_img(generated_images[0] * 255 , scale=False)
        img.save(os.path.join(save_dir,'generated_digit' + str(step) + '.png'))
        
        img = image.array_to_img(real_images[0] * 255 , scale=False)
        img.save(os.path.join(save_dir , 'real_digit' + str(step) + '.png'))
        

2022-02-08 15:13:33.078451: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8300


discriminator loss 0.44497770071029663
adverserial loss 0.8578011393547058
discriminator loss 0.006278226617723703
adverserial loss 6.564140319824219
discriminator loss 0.006830142345279455
adverserial loss 5.810070991516113
discriminator loss 0.006628791801631451
adverserial loss 8.101269721984863
discriminator loss 0.5787388682365417
adverserial loss 1.2627407312393188
discriminator loss 0.001227355794981122
adverserial loss 18.632516860961914
discriminator loss 0.007155274040997028
adverserial loss 9.259364128112793
discriminator loss 0.010220306925475597
adverserial loss 7.236536979675293
discriminator loss -0.0026436387561261654
adverserial loss 11.399665832519531
discriminator loss 0.5663478970527649
adverserial loss 1.0187572240829468
discriminator loss 0.0018093548715114594
adverserial loss 10.184630393981934
discriminator loss 0.033101048320531845
adverserial loss 9.688679695129395
discriminator loss 0.002028032438829541
adverserial loss 10.56730842590332
discriminator loss 0.